In [5]:
import torch
import safetensors as st
from deltazip import AutoDeltaZipModelForCausalLM, BaseCompressionConfig

In [3]:
tensors = {}
with st.safe_open(".local/compressed_models/lmsys.vicuna-7b-v1.5.2b75s128g/deltazip-compressed.safetensors", framework="torch", device="cpu") as f:
    for key in f.keys():
        tensors[key] = f.get_tensor(key)

In [7]:
compress_config = BaseCompressionConfig(
    bits=4,
    sparsity=0.3,
    prunen=0,
    block_size=128,
    prunem=0,
    lossless="gdeflate",
    damp_percent=0.0001,
    sym=False,
)
target_model_ref = AutoDeltaZipModelForCausalLM.from_pretrained(
    "lmsys/vicuna-7b-v1.5", 
    compress_config=compress_config,
    torch_dtype=torch.float16,
)

2024-04-07 17:00:57.727 | INFO     | deltazip.modeling._base:from_pretrained:699 - Using [None] to load model.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/home/xiayao/.local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.36s/it]
/mnt/scratch/xiayao/mamba/envs/deltazip/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initiali

In [9]:
missing_state_dict = target_model_ref.state_dict()
missing_state_dict = {
    k: v for k, v in missing_state_dict.items() if k not in tensors
}


{'lm_head.weight': tensor([  0,   0,   0,  ...,  78, 128, 103], dtype=torch.uint8), 'model.embed_tokens.weight': tensor([  0,   0,   0,  ..., 153, 183, 237], dtype=torch.uint8), 'model.layers.0.input_layernorm.weight': tensor([  0,   0,   0,  ...,  30, 141, 163], dtype=torch.uint8), 'model.layers.0.mlp.down_proj.g_idx': tensor([  0,   0,   0,  ..., 123, 199,  15], dtype=torch.uint8), 'model.layers.0.mlp.down_proj.qweight': tensor([  0,   0,   0,  ..., 227, 136, 246], dtype=torch.uint8), 'model.layers.0.mlp.down_proj.qzeros': tensor([  0,   0,   0,   0,   2,   2,   3,   0,   4,   3,   0,   0,   0,   0,
          0,   0,   0,   4,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,
          0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,
         88,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   4,   3,   0,   0,
 